In [2]:
import numpy as np
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from OrbitalRotation.tools.orbital_rotation_general import orbital_rotation_p_q, orbital_rotation_p_q_nf
from OrbitalRotation.tools.obt_to_tbt import get_obt_combined_tbt

In [3]:
from pyscf import gto, scf

mol = gto.M(atom="H 0 0 0; H 0 0 1", basis="sto3g", symmetry="d2h", verbose=0)
mf = scf.RHF(mol).run(conv_tol=1E-14)
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
    ncore=0, ncas=None, g2e_symm=8)
print(f"n Cas: {ncas}")
print(f"orb sym: {orb_sym}")
driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

n Cas: 2
orb sym: [0, 5]


In [3]:
print(h1e)

[[-1.06339037e+00 -1.28886187e-16]
 [ 7.08648939e-18 -6.14752718e-01]]


In [4]:
print(g2e)

[ 6.09171679e-01 -1.46384574e-17  2.03222227e-01  6.07335428e-01
  3.15485749e-17  6.37479877e-01]


In [9]:
orb_01 = orbital_rotation_p_q_nf(0, 1, 2, 0.1)

In [10]:
print(orb_01)

[[ 0.99500417  0.09983342]
 [-0.09983342  0.99500417]]


In [11]:
orb_01 @ h1e @ orb_01.conj().T

array([[-1.05891893,  0.04456527],
       [ 0.04456527, -0.61922416]])

# This is a simple test of optimizing a 2 by 2 one-body term

In [23]:
h1e = np.array([[-1.03, 0.4], [0.4, -0.67]])

In [24]:
from scipy.linalg import norm
from scipy.optimize import minimize

def cost_fn(theta):
    orb_01 = orbital_rotation_p_q_nf(0, 1, 2, theta)
    h_eff = orb_01 @ h1e @ orb_01.conj().T
    diag_h_eff = np.array([[h_eff[0, 0], 0], [0, h_eff[1, 1]]])
    return norm(h_eff - diag_h_eff, ord=None)

In [25]:
result = minimize(cost_fn, x0=np.array([0.1]))

In [26]:
print(result)

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 9.243525563498274e-09
        x: [-5.740e-01]
      nit: 2
      jac: [-9.575e-09]
 hess_inv: [[ 1.788e+00]]
     nfev: 52
     njev: 26


In [27]:
result.x[0]

-0.5739712077815586

In [28]:
original = cost_fn(0)
print(original)
minimized = cost_fn(result.x[0])
print(minimized)

0.5656854249492381
9.243525563498274e-09


# Simple test using a OBT combined TBT

In [31]:
combined_tbt = get_obt_combined_tbt(h1e, g2e, ncas, n_elec)

In [32]:
print(combined_tbt)

[[[[-0.82702527  0.2       ]
   [ 0.2        -0.65593857]]

  [[ 0.2         0.20322223]
   [ 0.20322223  0.2       ]]]


 [[[ 0.2         0.20322223]
   [ 0.20322223  0.2       ]]

  [[-0.65593857  0.2       ]
   [ 0.2        -0.45287117]]]]


In [4]:
import pyscf
from pathlib import Path 
mol = gto.M(atom="N 0 0 0; N 0 0 1.2", basis="sto3g", verbose=0)
mf = scf.RHF(mol).run(conv_tol=1E-14)
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
    ncore=0, ncas=None, g2e_symm=8)
print(f"n Cas: {ncas}")
print(f"orb sym: {orb_sym}")
driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

obt_dummy = np.zeros((ncas, ncas))
combined_tbt = get_obt_combined_tbt(h1e, g2e, ncas, n_elec)

### We use the chemist notation of TBT ###

label = "_tbt_combined"
filename = "fcidump.N2_12" + f"{label}"
pyscf.tools.fcidump.from_integrals(
    "fcidump_output"/Path(filename),
    obt_dummy,
    combined_tbt,
    nmo=obt_dummy.shape[0],
    nelec=n_elec,
    nuc=ecore,
    ms=spin,
    orbsym=None,
    tol=1E-8,
    float_format=' %.16g',
)

n Cas: 10
orb sym: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
